In [1]:
# =====================================================
# CELL 1: Install Dependencies
# =====================================================
"""
REALM RAG Pipeline for Autonomous Regulatory Compliance Auditor
----------------------------------------------------------------
This notebook implements a REALM-inspired RAG system for cross-jurisdictional
aviation safety compliance. Unlike standard RAG, REALM pre-trains the retriever
to understand domain-specific relationships between queries and documents.

Key Differences from Standard RAG:
1. Learned Relevance: Retriever is fine-tuned on domain-specific data
2. Probabilistic Retrieval: Uses P(z|x) to rank documents
3. Salient Span Masking: Pre-training forces retriever expertise
"""

!pip install -q langchain langchain-community langchain-core langchain-groq
!pip install -q sentence-transformers faiss-cpu pypdf groq
!pip install -q transformers torch datasets
!pip install -q scikit-learn pandas

In [3]:
# =====================================================
# CELL 2: Core Imports
# =====================================================
import os
import torch
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from IPython.display import display, Markdown

# LangChain 0.3 imports
from langchain_community.document_loaders import PyPDFLoader
# FIX: CharacterTextSplitter has moved to the 'langchain-text-splitters' package
!pip install -q langchain-text-splitters
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_groq import ChatGroq

# Transformer models
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.cross_encoder import CrossEncoder
from transformers import AutoTokenizer, AutoModel

print("All dependencies imported successfully!")

All dependencies imported successfully!


In [4]:
# =====================================================
# CELL 3: Configuration and Helper Classes
# =====================================================
@dataclass
class REALMConfig:
    """Configuration for REALM RAG pipeline"""
    chunk_size: int = 500
    chunk_overlap: int = 50
    top_k_retrieval: int = 10
    top_k_rerank: int = 5
    embedding_model: str = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
    reranker_model: str = "cross-encoder/ms-marco-MiniLM-L6-v2"
    llm_model: str = "llama-3.3-70b-versatile"
    temperature: float = 0.1

config = REALMConfig()

class REALMRetriever:
    """
    REALM-inspired retriever that implements probabilistic document selection.

    Key Features:
    - Dual-stage retrieval (dense + reranking)
    - Confidence scoring using P(z|x)
    - Domain-specific relevance learning
    """

    def __init__(self, vectorstore, reranker_model: str):
        self.vectorstore = vectorstore
        self.reranker = CrossEncoder(reranker_model)

    def retrieve_and_score(self, query: str, top_k: int = 10) -> List[Tuple]:
        """
        Retrieve documents with REALM-style probabilistic scoring.
        Returns: List of (document, relevance_score) tuples
        """
        # Stage 1: Dense retrieval using MIPS (Maximum Inner Product Search)
        initial_docs = self.vectorstore.similarity_search(query, k=top_k)

        # Stage 2: Rerank using cross-encoder (simulates P(z|x))
        pairs = [[query, doc.page_content] for doc in initial_docs]
        scores = self.reranker.predict(pairs)

        # Normalize scores to probabilities (softmax)
        exp_scores = np.exp(scores - np.max(scores))
        probabilities = exp_scores / exp_scores.sum()

        # Combine documents with their P(z|x) scores
        scored_docs = list(zip(initial_docs, probabilities))
        sorted_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)

        return sorted_docs

    def get_top_documents(self, query: str, top_k: int = 5):
        """Get top-k documents after reranking"""
        scored_docs = self.retrieve_and_score(query, top_k=config.top_k_retrieval)
        return [doc for doc, _ in scored_docs[:top_k]]

print("Configuration and helper classes defined!")

Configuration and helper classes defined!


In [6]:
# =====================================================
# CELL 4: Document Loading and Preprocessing
# =====================================================
"""
For this demo, we'll load actual aviation compliance documents from PDFs.
"""

from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks/AdvanceRAG Problems/REALM Data"
pdf_paths = ["CAAC_REALM_RAG.pdf", "FAA_REALM_RAG.pdf", "MSG3_REALM_RAG.pdf", "NTSB_REALM_RAG.pdf"]  # Replace with actual paths

documents = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    documents.extend(docs)

print(f"Loaded {len(documents)} compliance documents")
for doc in documents:
    print(f"  - {doc.metadata.get('source', 'Unknown')}: {len(doc.page_content)} chars")

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/AdvanceRAG Problems/REALM Data
Loaded 1095 compliance documents
  - CAAC_REALM_RAG.pdf: 2268 chars
  - CAAC_REALM_RAG.pdf: 1916 chars
  - CAAC_REALM_RAG.pdf: 2452 chars
  - CAAC_REALM_RAG.pdf: 2285 chars
  - CAAC_REALM_RAG.pdf: 2171 chars
  - CAAC_REALM_RAG.pdf: 2203 chars
  - CAAC_REALM_RAG.pdf: 2287 chars
  - CAAC_REALM_RAG.pdf: 2003 chars
  - CAAC_REALM_RAG.pdf: 2390 chars
  - CAAC_REALM_RAG.pdf: 2358 chars
  - CAAC_REALM_RAG.pdf: 2102 chars
  - CAAC_REALM_RAG.pdf: 2586 chars
  - CAAC_REALM_RAG.pdf: 2695 chars
  - CAAC_REALM_RAG.pdf: 2572 chars
  - CAAC_REALM_RAG.pdf: 2342 chars
  - CAAC_REALM_RAG.pdf: 2173 chars
  - CAAC_REALM_RAG.pdf: 1662 chars
  - FAA_REALM_RAG.pdf: 294 chars
  - FAA_REALM_RAG.pdf: 59 chars
  - FAA_REALM_RAG.pdf: 2844 chars
  - FAA_REALM_RAG.pdf: 2488 chars
  - FAA_REALM_RAG.pdf: 175 chars
  - FAA_REALM_RAG.pdf: 0 chars
  - FAA_REALM_RAG.pdf: 3139 chars
  - FAA_REALM_RAG.pdf: 0 chars
  - FAA_REALM_

In [7]:
# =====================================================
# CELL 5: Text Chunking with Overlap
# =====================================================
"""
REALM requires careful chunking to preserve regulatory context.
We use CharacterTextSplitter to maintain semantic coherence.
"""

text_splitter = CharacterTextSplitter(
    chunk_size=config.chunk_size,
    chunk_overlap=config.chunk_overlap,
)

chunks = text_splitter.split_documents(documents)

# Add chunk metadata
for i, chunk in enumerate(chunks):
    chunk.metadata["chunk_id"] = i

print(f"✅ Created {len(chunks)} chunks from {len(documents)} documents")
print(f"   Chunk size: {config.chunk_size} chars | Overlap: {config.chunk_overlap} chars")

# Preview first chunk
print("\n📄 Sample Chunk:")
print(f"Source: {chunks[0].metadata.get('source', 'N/A')}")
print(f"Content: {chunks[0].page_content[:300]}...")

✅ Created 1041 chunks from 1095 documents
   Chunk size: 500 chars | Overlap: 50 chars

📄 Sample Chunk:
Source: CAAC_REALM_RAG.pdf
Content: Translation for Reference Only
Regulation on the Management of PublicAir Transportation Services
for Passengers
Chapter 1 General Principles
Article 1 In order to strengthen the management of public air transport services for
passengers, protect the legitimate rights and interests of passengers and ...


In [8]:
# =====================================================
# CELL 6: Initialize Embeddings (REALM-style Dense Retrieval)
# =====================================================
"""
REALM uses a BERT-based encoder for dense retrieval.
We use multi-qa-mpnet which is optimized for question-answering retrieval.

Key: This model has been pre-trained on MS MARCO and Natural Questions,
simulating REALM's Wikipedia-scale pre-training.
"""

embedding_model = HuggingFaceEmbeddings(
    model_name=config.embedding_model,
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
    encode_kwargs={'normalize_embeddings': True}  # For cosine similarity
)

print(f"✅ Initialized embeddings: {config.embedding_model}")
print(f"   Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# Test embedding
test_embedding = embedding_model.embed_query("What are the fire suppression requirements?")
print(f"   Embedding dimension: {len(test_embedding)}")

/tmp/ipython-input-58416617.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Initialized embeddings: sentence-transformers/multi-qa-mpnet-base-dot-v1
   Device: GPU
   Embedding dimension: 768


In [9]:
# =====================================================
# CELL 7: Create FAISS Vector Store (MIPS Index)
# =====================================================
"""
FAISS implements Maximum Inner Product Search (MIPS), which is core to REALM.
This allows efficient retrieval from large knowledge corpora.
"""

vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

print(f"✅ FAISS vector store created with {len(chunks)} vectors")
print(f"   Index type: MIPS (Maximum Inner Product Search)")

# Test similarity search
test_query = "N1 vibration limits"
test_results = vectorstore.similarity_search(test_query, k=3)
print(f"\n🔍 Test Query: '{test_query}'")
print(f"   Retrieved {len(test_results)} documents")

✅ FAISS vector store created with 1041 vectors
   Index type: MIPS (Maximum Inner Product Search)

🔍 Test Query: 'N1 vibration limits'
   Retrieved 3 documents


In [10]:
# =====================================================
# CELL 8: Initialize REALM Retriever
# =====================================================
"""
The REALMRetriever implements the two-stage retrieval process:
1. Dense retrieval (MIPS) - fast, recalls top-k candidates
2. Cross-encoder reranking - slow but accurate, simulates P(z|x)
"""

realm_retriever = REALMRetriever(
    vectorstore=vectorstore,
    reranker_model=config.reranker_model
)

print(f"✅ REALM Retriever initialized")
print(f"   Stage 1: Dense retrieval (top-{config.top_k_retrieval})")
print(f"   Stage 2: Cross-encoder reranking (top-{config.top_k_rerank})")

# Test retrieval with scoring
test_query = "What modifications are required when transferring from FAA to EASA registry?"
scored_results = realm_retriever.retrieve_and_score(test_query, top_k=5)

print(f"\n🎯 Test Retrieval with P(z|x) Scores:")
print(f"Query: {test_query}\n")
for i, (doc, prob) in enumerate(scored_results[:3], 1):
    print(f"Rank {i} | P(z|x) = {prob:.4f}")
    print(f"Source: {doc.metadata.get('source', 'N/A')}")
    print(f"Content: {doc.page_content[:200]}...\n")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ REALM Retriever initialized
   Stage 1: Dense retrieval (top-10)
   Stage 2: Cross-encoder reranking (top-5)

🎯 Test Retrieval with P(z|x) Scores:
Query: What modifications are required when transferring from FAA to EASA registry?

Rank 1 | P(z|x) = 0.6198
Source: FAA_REALM_RAG.pdf
Content: AIM 4/20/23 
TBL 4−1 
Flight Specific Information 
Item International Flight 
Plan (FAA Form 7233−4) Domestic U.S. Requirements 
Equivalent Item on 
Domestic Flight Plan 
(FAA Form 7233−1) 
Aircraft I...

Rank 2 | P(z|x) = 0.3247
Source: FAA_REALM_RAG.pdf
Content: AIM 4/20/23 
TBL 4−19 
Aircraft Specific Information 
Item 
International Flight Plan 
(FAA Form 7233−4) Domestic U.S. Requirements 
Equivalent Item on 
Domestic Flight Plan 
(FAA Form 7233−1) 
Number...

Rank 3 | P(z|x) = 0.0352
Source: FAA_REALM_RAG.pdf
Content: AIM 4/20/23 
j. 11 −1−1. GENERAL 
11−1−2. ACCESS TO THE NATIONAL AIRSPACE SYSTEM (NAS) FOR UAS OPERATORS 
11−2−1. PART 107 SMALL UAS AND RECREATIONAL FLYERS 
11−2−2. REGISTRATI

In [11]:
# =====================================================
# CELL 9: Initialize LLM with Groq
# =====================================================
"""
Configure the LLM for generating compliance answers.
Low temperature ensures factual, deterministic responses.
"""

from google.colab import userdata

llm = ChatGroq(
    model=config.llm_model,
    temperature=config.temperature,
    api_key=userdata.get('GROQ_API_KEY')  # Set in Colab Secrets
)

print(f"✅ LLM initialized: {config.llm_model}")
print(f"   Temperature: {config.temperature} (deterministic)")

# Test LLM
test_response = llm.invoke([HumanMessage(content="Say 'LLM ready' if you can read this.")])
print(f"   Status: {test_response.content}")

✅ LLM initialized: llama-3.3-70b-versatile
   Temperature: 0.1 (deterministic)
   Status: LLM ready


In [12]:
# =====================================================
# CELL 10: Create REALM-Style Prompt Template
# =====================================================
"""
REALM prompt emphasizes:
1. Multi-document reasoning (cross-jurisdictional analysis)
2. Explicit citation of source regulations
3. Highlighting contradictions between jurisdictions
"""

realm_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are an expert aviation compliance auditor analyzing multi-jurisdictional regulations.

Your task is to:
1. **Identify Exact Regulatory Requirements** from FAA, EASA, and CAAC documents
2. **Highlight Differences** between jurisdictions (e.g., different agent types, timing, thresholds)
3. **Cite Specific Sections** (e.g., "per FAA Part 25.851" or "EASA CS-25 requires...")
4. **Assess Compliance Risk** - which regulation is most restrictive?
5. **Provide Actionable Guidance** for cross-registry transfers

**Critical**: Base your answer ONLY on the provided context. If regulations conflict, state both requirements clearly."""),

    HumanMessage(content="""**Retrieved Regulatory Context:**

{context}

---

**Compliance Query:**
{question}

**Analysis Instructions:**
- Compare FAA, EASA, and CAAC requirements if all are present
- Identify the most restrictive standard (this applies per ICAO rules)
- Note any required modifications for registry transfers
- Cite specific sections and standards

**Your Expert Analysis:**""")
])

print("✅ REALM prompt template created")
print("   Focus: Multi-jurisdictional regulatory analysis")

✅ REALM prompt template created
   Focus: Multi-jurisdictional regulatory analysis


In [13]:
# =====================================================
# CELL 11: Build REALM RAG Chain (LangChain 0.3 Runnables)
# =====================================================
"""
LangChain 0.3 uses LCEL (LangChain Expression Language) with Runnables.
This chain implements the REALM pipeline: Retrieve → Rerank → Generate
"""

def format_docs(docs):
    """Format retrieved documents with metadata for context"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        section = doc.metadata.get('section', '')
        jurisdiction = doc.metadata.get('jurisdiction', '')

        header = f"[Document {i}: {source}"
        if section:
            header += f" §{section}"
        if jurisdiction:
            header += f" ({jurisdiction})"
        header += "]"

        formatted.append(f"{header}\n{doc.page_content}\n")

    return "\n---\n".join(formatted)

# Custom retriever function that uses REALM retriever
def realm_retrieve(query: str):
    """REALM-style retrieval with probabilistic scoring"""
    return realm_retriever.get_top_documents(query, top_k=config.top_k_rerank)

# Build the REALM RAG chain using LCEL
realm_rag_chain = (
    {
        "context": lambda x: format_docs(realm_retrieve(x["question"])),
        "question": lambda x: x["question"]
    }
    | realm_prompt
    | llm
    | StrOutputParser()
)

print("✅ REALM RAG Chain constructed using LangChain 0.3 Runnables")
print("   Pipeline: Query → MIPS Retrieval → Cross-Encoder Rerank → LLM Generation")

✅ REALM RAG Chain constructed using LangChain 0.3 Runnables
   Pipeline: Query → MIPS Retrieval → Cross-Encoder Rerank → LLM Generation


In [14]:
# =====================================================
# CELL 12: Test Query 1 - Cross-Jurisdictional Analysis
# =====================================================
"""
Test Case 1: The exact scenario from your requirements.
This tests REALM's ability to identify delta between jurisdictions.
"""

query_1 = """We are moving an aircraft from a US (FAA) registry to a French (EASA) registry.
What specific modifications are required for the fire suppression system under EASA CS-25
that differ from FAA Part 25?"""

print("🔍 QUERY 1: Cross-Jurisdictional Registry Transfer\n")
print(f"Question: {query_1}\n")
print("="*80)

# Show retrieved documents with scores
print("\n📚 REALM Retrieval Process:\n")
scored_docs = realm_retriever.retrieve_and_score(query_1, top_k=config.top_k_retrieval)
print(f"Stage 1: Retrieved {len(scored_docs)} candidate documents")
print(f"\nStage 2: Top-{config.top_k_rerank} after reranking (with P(z|x) scores):\n")

for i, (doc, prob) in enumerate(scored_docs[:config.top_k_rerank], 1):
    print(f"Rank {i} | P(z|x) = {prob:.4f} | Source: {doc.metadata.get('source', 'N/A')}")

# Generate answer
print("\n" + "="*80)
print("\n🤖 REALM RAG ANSWER:\n")

response_1 = realm_rag_chain.invoke({"question": query_1})
display(Markdown(response_1))

🔍 QUERY 1: Cross-Jurisdictional Registry Transfer

Question: We are moving an aircraft from a US (FAA) registry to a French (EASA) registry.
What specific modifications are required for the fire suppression system under EASA CS-25
that differ from FAA Part 25?


📚 REALM Retrieval Process:

Stage 1: Retrieved 10 candidate documents

Stage 2: Top-5 after reranking (with P(z|x) scores):

Rank 1 | P(z|x) = 0.3184 | Source: FAA_REALM_RAG.pdf
Rank 2 | P(z|x) = 0.3023 | Source: FAA_REALM_RAG.pdf
Rank 3 | P(z|x) = 0.1379 | Source: FAA_REALM_RAG.pdf
Rank 4 | P(z|x) = 0.0839 | Source: FAA_REALM_RAG.pdf
Rank 5 | P(z|x) = 0.0731 | Source: FAA_REALM_RAG.pdf


🤖 REALM RAG ANSWER:



Given the lack of specific regulatory context and compliance query in your request, I will provide a general framework for analyzing multi-jurisdictional regulations based on typical requirements from the Federal Aviation Administration (FAA), European Union Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). This framework will cover the identification of exact regulatory requirements, highlighting differences, citing specific sections, assessing compliance risk, and providing actionable guidance for cross-registry transfers.

### 1. Identify Exact Regulatory Requirements

- **FAA Requirements**: For example, in the context of aircraft design and airworthiness, the FAA has specific regulations outlined in Part 25 of the Federal Aviation Regulations (FARs), which pertains to transport category airplanes. Per FAA Part 25.851, requirements for fire protection in aircraft are detailed.
- **EASA Requirements**: EASA CS-25, similar to FAA Part 25, outlines the airworthiness standards for large commercial airplanes. EASA CS-25 requires specific standards for fire protection, which might differ slightly from FAA requirements.
- **CAAC Requirements**: The CAAC has its own set of regulations, which may align closely with either FAA or EASA standards, or may have unique requirements. For instance, CAAC might have specific regulations regarding the materials used in aircraft construction to meet local environmental standards.

### 2. Highlight Differences

- **Agent Types**: Different jurisdictions might recognize different types of agents for representation in regulatory matters. For example, the FAA might require a specific type of legal representative for foreign operators, whereas EASA might accept a broader range of representatives.
- **Timing and Thresholds**: The timing for compliance with certain regulations and the thresholds that trigger regulatory requirements can vary significantly between jurisdictions. For instance, the lead time for submitting certain documents or the passenger/cargo thresholds that necessitate specific safety equipment can differ.

### 3. Cite Specific Sections

- **FAA Part 25.851** details the fire protection requirements for aircraft, emphasizing the importance of protecting both the passengers and the aircraft structure from fire hazards.
- **EASA CS-25.851** similarly focuses on fire protection but might include additional or slightly different requirements, reflecting EASA's approach to safety.
- **CAAC Regulations**, while potentially less detailed in some areas compared to FAA or EASA, would still outline specific requirements for fire safety, possibly with an emphasis on materials and construction methods approved by the CAAC.

### 4. Assess Compliance Risk

The most restrictive standard, as per ICAO rules, would typically be the one that an operator must adhere to when operating in multiple jurisdictions. For example, if an aircraft is registered in the United States but operates frequently in Europe, it would need to comply with both FAA and EASA regulations, adhering to the most stringent requirements of both.

### 5. Provide Actionable Guidance

For cross-registry transfers, the following steps are recommended:
- **Review Regulatory Requirements**: Thoroughly review the regulations of both the current and intended registry jurisdictions.
- **Assess Aircraft Compliance**: Evaluate the aircraft's current configuration against the regulatory requirements of the intended registry.
- **Plan Modifications**: Identify any necessary modifications to achieve compliance with the new jurisdiction's regulations.
- **Consult with Authorities**: Engage with both the current and intended registry authorities to clarify any questions and ensure a smooth transfer process.
- **Execute Modifications and Transfer**: Complete any required modifications and then proceed with the registry transfer, ensuring all documentation is in order and submitted as required.

In conclusion, while the exact regulatory context and compliance query are not provided, this framework offers a structured approach to analyzing and complying with multi-jurisdictional aviation regulations. It emphasizes the importance of understanding the specific requirements of each jurisdiction involved and adhering to the most restrictive standards to ensure compliance and safety.

In [15]:
# =====================================================
# CELL 13: Test Query 2 - Technical Specification Lookup
# =====================================================
"""
Test Case 2: Testing REALM's learned relevance for technical terms.
"N1 Vibration Limit" should retrieve the right document even though
the query and document don't have high lexical overlap.
"""

query_2 = """What is the N1 vibration limit during fire suppression system activation,
and how does it differ between FAA and CAAC regulations?"""

print("\n" + "="*80)
print("\n🔍 QUERY 2: Technical Specification Comparison\n")
print(f"Question: {query_2}\n")
print("="*80)

# Show retrieval
print("\n📚 Retrieved Documents:\n")
scored_docs_2 = realm_retriever.retrieve_and_score(query_2, top_k=config.top_k_retrieval)

for i, (doc, prob) in enumerate(scored_docs_2[:3], 1):
    print(f"\nRank {i} | P(z|x) = {prob:.4f}")
    print(f"Source: {doc.metadata.get('source', 'N/A')}")
    print(f"Preview: {doc.page_content[:250]}...")

# Generate answer
print("\n" + "="*80)
print("\n🤖 REALM RAG ANSWER:\n")

response_2 = realm_rag_chain.invoke({"question": query_2})
display(Markdown(response_2))



🔍 QUERY 2: Technical Specification Comparison

Question: What is the N1 vibration limit during fire suppression system activation,
and how does it differ between FAA and CAAC regulations?


📚 Retrieved Documents:


Rank 1 | P(z|x) = 0.5766
Source: FAA_REALM_RAG.pdf
Preview: AIM 4/20/233/15/077110.65R CHG 2AIM 10/5/23 
but are not limited to: toxic gas leaks or spills, flammable agents, or fumes which if fanned by rotor or propeller 
wash could endanger persons or property on the surface, or if entered by an aircraft cou...

Rank 2 | P(z|x) = 0.1484
Source: FAA_REALM_RAG.pdf
Preview: 4/20/23 AIM 
Section 6. National Security and Interception Procedures 
5−6−1. National Security 
National security in the control of air traffic is governed by 14 Code of Federal Regulations (CFR) Part 99, 
Security Control of Air Traffic. 
5−6−2. Na...

Rank 3 | P(z|x) = 0.0780
Source: NTSB_REALM_RAG.pdf
Preview: NTSB Aircraft Accident Report 
24
1.8.3 Events Involving Inadvertent Go-A round Mode Activat

Given the lack of specific regulatory context and compliance query in your request, I will provide a general framework for analyzing multi-jurisdictional regulations in aviation, focusing on the Federal Aviation Administration (FAA), European Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). This framework will outline how to identify exact regulatory requirements, highlight differences, cite specific sections, assess compliance risk, and provide actionable guidance for cross-registry transfers.

### 1. Identify Exact Regulatory Requirements

- **FAA Requirements**: For aircraft design and airworthiness, the FAA often references Part 25 of the Federal Aviation Regulations (FARs). For example, fire protection requirements can be found in Part 25.851, which outlines the standards for the protection of the aircraft against fire.
- **EASA Requirements**: EASA's Certification Specifications for Large Aeroplanes (CS-25) provide detailed requirements for airworthiness. CS-25 also covers fire protection, among other aspects, ensuring that aircraft are designed to minimize the risk of fire and its effects.
- **CAAC Requirements**: The CAAC has its own set of regulations, often aligning with international standards but also including specific national requirements. For fire protection, CAAC might reference CCAR-25 (China Civil Aviation Regulations), which would be similar to but distinct from FAA Part 25 and EASA CS-25.

### 2. Highlight Differences

- **Agent Types**: Different jurisdictions may have varying requirements for fire extinguishing agents. For example, the FAA might allow certain types of halon substitutes, while EASA could have stricter regulations due to environmental concerns, and CAAC might have its own list of approved agents.
- **Timing and Thresholds**: The timing for compliance with new regulations and the thresholds for when certain regulations apply can differ. For instance, EASA might require compliance with a new standard within a shorter timeframe compared to the FAA.

### 3. Cite Specific Sections

- **FAA Part 25.851**: This section specifically deals with fire protection requirements for aircraft, including the use of fire-resistant materials and the installation of fire detectors and extinguishing systems.
- **EASA CS-25.851**: Similar to the FAA's requirement, EASA's CS-25 outlines the fire protection standards, including the performance requirements for fire-resistant materials and the need for an approved fire extinguishing system.
- **CAAC CCAR-25**: While specific sections might not directly correlate with FAA or EASA regulations, CCAR-25 would contain China's standards for fire protection, potentially including unique requirements based on national regulations.

### 4. Assess Compliance Risk

The most restrictive standard, as per ICAO rules, would typically be the one that an operator must comply with to ensure safety and avoid regulatory issues. If, for example, EASA has stricter fire protection requirements than the FAA and CAAC, an aircraft operator planning to register an aircraft in Europe would need to comply with EASA's standards, even if the aircraft was previously registered elsewhere.

### 5. Provide Actionable Guidance for Cross-Registry Transfers

For an aircraft being transferred from one registry to another (e.g., from the FAA registry to the EASA registry), the following steps are recommended:
- **Conduct a Gap Analysis**: Identify the differences in regulatory requirements between the current and future registries.
- **Modify the Aircraft as Necessary**: Ensure the aircraft complies with the most restrictive standards of the new registry. This might involve installing new fire extinguishing systems, replacing materials, or updating electrical systems.
- **Obtain New Certifications**: After modifications, the aircraft must be certified by the new regulatory authority. This involves inspections and potentially new flight tests to ensure compliance with local regulations.
- **Register the Aircraft**: Finally, complete the registration process with the new authority, ensuring all paperwork and regulatory requirements are met.

In conclusion, when dealing with multi-jurisdictional regulations in aviation, it's crucial to identify the exact requirements of each relevant authority, highlight any differences, and comply with the most restrictive standards to minimize compliance risk. Always referencing the specific sections of the regulations (e.g., FAA Part 25, EASA CS-25) and understanding the requirements for cross-registry transfers are key to successful compliance and operation.

In [16]:
# =====================================================
# CELL 14: Test Query 3 - Maintenance History Analysis
# =====================================================
"""
Test Case 3: Cross-referencing maintenance logs with accident reports.
Tests REALM's ability to connect historical data with current regulations.
"""

query_3 = """Based on NTSB accident reports, what changes were made to fire suppression
inspection intervals, and do current MSG-3 procedures reflect these changes?"""

print("\n" + "="*80)
print("\n🔍 QUERY 3: Regulatory Change History Analysis\n")
print(f"Question: {query_3}\n")
print("="*80)

response_3 = realm_rag_chain.invoke({"question": query_3})
display(Markdown(response_3))



🔍 QUERY 3: Regulatory Change History Analysis

Question: Based on NTSB accident reports, what changes were made to fire suppression
inspection intervals, and do current MSG-3 procedures reflect these changes?



Given the lack of specific regulatory context and compliance query in your request, I'll provide a general framework for analyzing multi-jurisdictional regulations based on typical requirements from the Federal Aviation Administration (FAA), European Union Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). This framework will cover the identification of exact regulatory requirements, highlighting differences, citing specific sections, assessing compliance risk, and providing actionable guidance for cross-registry transfers.

### 1. Identify Exact Regulatory Requirements

- **FAA Requirements**: Typically, the FAA outlines its requirements in various parts of the Federal Aviation Regulations (FARs). For example, Part 25 contains airworthiness standards for transport category airplanes. Per FAA Part 25.851, specific fire protection requirements must be met.
- **EASA Requirements**: EASA's regulations can be found in the Certification Specifications (CS) and Acceptable Means of Compliance (AMC). For instance, EASA CS-25 requires compliance with specific airworthiness standards for large airplanes, including those related to fire protection.
- **CAAC Requirements**: The CAAC has its own set of regulations and standards, often aligning with international norms but with specific requirements for operations within China. These might include unique identifiers for aircraft registered in China.

### 2. Highlight Differences

- **Agent Types**: Different jurisdictions may recognize different types of agents or representatives for foreign operators. For example, the FAA might require a specific type of agent for U.S.-registered aircraft operating internationally.
- **Timing and Thresholds**: The timing for compliance and the thresholds for applicability can vary. For instance, EASA might have a more stringent timeline for implementing certain safety regulations compared to the FAA.
- **Registry Transfer Requirements**: The process and requirements for transferring an aircraft's registry from one jurisdiction to another can be complex and vary significantly. This might involve demonstrating compliance with the new jurisdiction's regulations, possibly requiring modifications to the aircraft.

### 3. Cite Specific Sections

- **FAA Part 25.851** deals with fire protection requirements for transport category airplanes.
- **EASA CS-25** outlines the airworthiness standards for large airplanes, including requirements for structures, performance, and equipment.
- **CAAC Regulations** might cite specific sections related to Chinese aviation law and regulations, such as those dealing with aircraft registration, airworthiness, and operational requirements.

### 4. Assess Compliance Risk

The most restrictive standard, as per ICAO rules, would typically be the one that an operator must adhere to, especially in cases of cross-registry transfers or international operations. For example, if an aircraft is being transferred from an EASA registry to an FAA registry, the operator must ensure compliance with the more restrictive of the two sets of regulations, which could involve additional modifications or inspections.

### 5. Provide Actionable Guidance

For cross-registry transfers, the following steps are recommended:
- **Conduct a Gap Analysis**: Identify the differences between the current and target registry's regulatory requirements.
- **Modify the Aircraft as Necessary**: Ensure the aircraft complies with the more restrictive standards, which may involve physical modifications, updating documentation, or obtaining new certifications.
- **Engage with Regulatory Authorities**: Work closely with both the current and target registry's aviation authorities to understand their specific requirements and to ensure a smooth transfer process.
- **Update Documentation**: Reflect the changes in the aircraft's registration and compliance status in all relevant documents, including the aircraft's certificate of airworthiness and operational manuals.

Without specific details on the regulatory context or compliance query, this analysis provides a general approach to navigating multi-jurisdictional aviation regulations. For precise guidance, the exact requirements of the jurisdictions involved must be consulted.

In [17]:
# =====================================================
# CELL 15: Comparative Analysis - Standard RAG vs REALM
# =====================================================
"""
Compare standard RAG (no reranking) with REALM RAG (with reranking).
This demonstrates the value of the REALM approach.
"""

print("\n" + "="*80)
print("📊 COMPARATIVE ANALYSIS: Standard RAG vs REALM RAG")
print("="*80)

test_query = "Registry transfer fire suppression requirements"

# Standard RAG (just vector similarity)
print("\n1️⃣ STANDARD RAG (Vector Similarity Only):\n")
standard_docs = vectorstore.similarity_search(test_query, k=3)
for i, doc in enumerate(standard_docs, 1):
    print(f"Rank {i} | Source: {doc.metadata.get('source', 'N/A')}")

# REALM RAG (with reranking)
print("\n2️⃣ REALM RAG (With Cross-Encoder Reranking):\n")
realm_docs = realm_retriever.retrieve_and_score(test_query, top_k=5)
for i, (doc, prob) in enumerate(realm_docs[:3], 1):
    print(f"Rank {i} | P(z|x) = {prob:.4f} | Source: {doc.metadata.get('source', 'N/A')}")

print("\n💡 Key Insight: REALM reranking often changes document order significantly,")
print("   retrieving documents that are semantically relevant but lexically distant.")


📊 COMPARATIVE ANALYSIS: Standard RAG vs REALM RAG

1️⃣ STANDARD RAG (Vector Similarity Only):

Rank 1 | Source: FAA_REALM_RAG.pdf
Rank 2 | Source: FAA_REALM_RAG.pdf
Rank 3 | Source: FAA_REALM_RAG.pdf

2️⃣ REALM RAG (With Cross-Encoder Reranking):

Rank 1 | P(z|x) = 0.3140 | Source: FAA_REALM_RAG.pdf
Rank 2 | P(z|x) = 0.2234 | Source: FAA_REALM_RAG.pdf
Rank 3 | P(z|x) = 0.1730 | Source: FAA_REALM_RAG.pdf

💡 Key Insight: REALM reranking often changes document order significantly,
   retrieving documents that are semantically relevant but lexically distant.


In [18]:
# =====================================================
# CELL 16: Interactive Query Interface
# =====================================================
"""
Interactive interface for testing your own compliance queries.
"""

def query_realm_system(question: str, show_sources: bool = True):
    """
    Query the REALM RAG system with detailed output.

    Args:
        question: Compliance query
        show_sources: Whether to display source documents
    """
    print("🔍 QUERY:", question)
    print("="*80)

    if show_sources:
        print("\n📚 Retrieved Sources (with REALM scores):\n")
        scored_docs = realm_retriever.retrieve_and_score(question, top_k=config.top_k_retrieval)
        for i, (doc, prob) in enumerate(scored_docs[:config.top_k_rerank], 1):
            print(f"\n[{i}] P(z|x) = {prob:.4f} | {doc.metadata.get('source', 'N/A')}")
            print(f"    {doc.page_content[:200]}...")

    print("\n" + "="*80)
    print("🤖 ANSWER:\n")

    response = realm_rag_chain.invoke({"question": question})
    display(Markdown(response))

    return response

# Example usage:
print("✅ Interactive query function ready!")
print("\nExample usage:")
print("```python")
print('query_realm_system("Your compliance question here")')
print("```")

# Test it:
query_realm_system(
    "What are the hydrostatic testing requirements for fire suppression bottles?",
    show_sources=True
)

✅ Interactive query function ready!

Example usage:
```python
query_realm_system("Your compliance question here")
```
🔍 QUERY: What are the hydrostatic testing requirements for fire suppression bottles?

📚 Retrieved Sources (with REALM scores):


[1] P(z|x) = 0.3118 | FAA_REALM_RAG.pdf
    AIM 4/20/233/15/077110.65R CHG 2AIM 3/21/24 
e. It is important to remember that precipitation static problems can only be corrected with the proper number 
of quality static dischargers, properly ins...

[2] P(z|x) = 0.1637 | FAA_REALM_RAG.pdf
    AIM 4/20/23 
(2) If shutdown on a helideck, request the supervisor in charge provide a briefing on location of 
protective equipment and safety procedures. 
(3) If while flying near a helideck and the...

[3] P(z|x) = 0.0808 | FAA_REALM_RAG.pdf
    AIM 4/20/23 
NOTE− 
The F AA’s Advanced Operations website may be viewed at: https://www.faa.gov/uas/advanced_operations/ . 
(b) Advisory Circular 137−1, Certification Process for Agricultural Aircraf...

[4] P(

Given the absence of specific regulatory context and a compliance query in your request, I will provide a general framework for analyzing multi-jurisdictional regulations based on typical requirements from the Federal Aviation Administration (FAA), European Union Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). This framework will cover the identification of exact regulatory requirements, highlighting differences between jurisdictions, citing specific sections, assessing compliance risk, and providing actionable guidance for cross-registry transfers.

### Identify Exact Regulatory Requirements

1. **FAA Requirements**: For example, per FAA Part 25.851, specific standards for flight compartment instrumentation are outlined. Additionally, the FAA has regulations regarding aircraft registration, airworthiness, and operational requirements that must be met for both domestic and international operations.

2. **EASA Requirements**: EASA CS-25 requires specific design and performance standards for large aeroplanes. EASA also has detailed regulations on continuing airworthiness, maintenance requirements, and environmental protection.

3. **CAAC Requirements**: The CAAC has its own set of regulations, including those related to aircraft registration, certification, and operation within Chinese airspace. These regulations may include specific requirements for equipment, crew training, and maintenance.

### Highlight Differences

- **Agent Types and Responsibilities**: Different jurisdictions may recognize different types of agents (e.g., continuing airworthiness management organizations) with varying responsibilities.
- **Timing and Thresholds**: Requirements for inspection intervals, maintenance thresholds, and compliance timelines can differ significantly between the FAA, EASA, and CAAC.
- **Environmental and Noise Regulations**: EASA and the FAA have strict regulations regarding noise and emissions, which might be more stringent than those in other jurisdictions.

### Cite Specific Sections

- **FAA Part 121** outlines operational requirements for airlines, including those related to safety management systems and crew training.
- **EASA Part-M** covers continuing airworthiness requirements, including maintenance programs and airworthiness review.
- **CAAC Regulations** (e.g., CCAR-25) would outline specific standards for the design, production, and operation of aircraft in China.

### Assess Compliance Risk

The most restrictive standard, as per ICAO rules, would typically be followed to ensure compliance across jurisdictions. For instance, if EASA has more stringent noise reduction requirements than the FAA or CAAC, an aircraft operator intending to fly in EASA-regulated airspace would need to comply with EASA's standards.

### Provide Actionable Guidance

For cross-registry transfers, the following steps are recommended:
1. **Review Regulatory Requirements**: Ensure compliance with the most restrictive standards applicable to the jurisdictions involved.
2. **Conduct a Gap Analysis**: Identify any differences in requirements between the current registry and the intended registry.
3. **Implement Necessary Modifications**: Make any required modifications to the aircraft, its documentation, or operational procedures to comply with the new jurisdiction's regulations.
4. **Obtain Necessary Approvals**: Secure approvals from the relevant aviation authorities for the transfer, including any required certifications or inspections.

Without specific context or a query, this analysis provides a general approach to navigating multi-jurisdictional aviation regulations. For precise guidance, the exact regulatory context and compliance query would need to be provided.

"Given the absence of specific regulatory context and a compliance query in your request, I will provide a general framework for analyzing multi-jurisdictional regulations based on typical requirements from the Federal Aviation Administration (FAA), European Union Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). This framework will cover the identification of exact regulatory requirements, highlighting differences between jurisdictions, citing specific sections, assessing compliance risk, and providing actionable guidance for cross-registry transfers.\n\n### Identify Exact Regulatory Requirements\n\n1. **FAA Requirements**: For example, per FAA Part 25.851, specific standards for flight compartment instrumentation are outlined. Additionally, the FAA has regulations regarding aircraft registration, airworthiness, and operational requirements that must be met for both domestic and international operations.\n\n2. **EASA Requirements**: EASA CS-25 requires 

In [19]:
# =====================================================
# CELL 17: Batch Query Analysis
# =====================================================
"""
Run multiple queries in batch to analyze system performance.
"""

batch_queries = [
    "What is the maximum activation time for fire suppression systems?",
    "Which jurisdiction has the most restrictive N1 vibration limits?",
    "What fire suppression agent is required under EASA regulations?",
    "What inspection intervals apply to fire suppression systems?"
]

print("📋 BATCH QUERY ANALYSIS")
print("="*80)

results = []
for i, query in enumerate(batch_queries, 1):
    print(f"\n[Query {i}/{len(batch_queries)}] {query}")
    response = realm_rag_chain.invoke({"question": query})
    results.append({"query": query, "response": response})
    print(f"✅ Completed\n")

# Display all results
print("\n" + "="*80)
print("📊 ALL RESULTS:")
print("="*80)

for i, result in enumerate(results, 1):
    print(f"\n{'='*80}")
    print(f"QUERY {i}: {result['query']}")
    print(f"{'='*80}\n")
    display(Markdown(result['response']))

📋 BATCH QUERY ANALYSIS

[Query 1/4] What is the maximum activation time for fire suppression systems?
✅ Completed


[Query 2/4] Which jurisdiction has the most restrictive N1 vibration limits?
✅ Completed


[Query 3/4] What fire suppression agent is required under EASA regulations?
✅ Completed


[Query 4/4] What inspection intervals apply to fire suppression systems?
✅ Completed


📊 ALL RESULTS:

QUERY 1: What is the maximum activation time for fire suppression systems?



Given the absence of specific regulatory context and a compliance query in your request, I will provide a general framework for analyzing multi-jurisdictional regulations for aviation compliance, focusing on the Federal Aviation Administration (FAA), European Union Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). This framework will highlight how to identify exact regulatory requirements, differences between jurisdictions, cite specific sections, assess compliance risk, and provide actionable guidance for cross-registry transfers.

### Identifying Exact Regulatory Requirements

1. **FAA Requirements**: For issues related to aircraft design, production, and airworthiness, refer to the Federal Aviation Regulations (FARs), such as Part 25 for transport category airplanes. For example, fire protection requirements can be found in "FAA Part 25.851" which outlines the standards for the protection of the airplane against fire.

2. **EASA Requirements**: EASA's Certification Specifications (CS) provide the regulatory framework. For instance, "EASA CS-25" is the certification specification for large airplanes, detailing requirements for airworthiness, including structural integrity and performance.

3. **CAAC Requirements**: The CAAC has its own set of regulations and standards, often aligned with international standards but with specific national requirements. For example, the "CCAR-25" (Chinese Civil Aviation Regulations) mirrors many aspects of the FAA's Part 25 but may include additional or differing requirements.

### Highlighting Differences

- **Agent Types and Responsibilities**: Differences may exist in the roles and responsibilities of agents or representatives in each jurisdiction, affecting how compliance is managed and reported.
- **Timing and Thresholds**: Various jurisdictions may have different timelines for compliance with new regulations or different thresholds for when certain regulations apply (e.g., weight, passenger capacity).

### Citing Specific Sections

- **FAA Part 25.851** deals with fire protection, emphasizing the importance of protecting areas where flammable fluids or vapors might be present.
- **EASA CS-25.851** similarly addresses fire protection but may include additional requirements or nuances not present in the FAA regulations.
- **CCAR-25** would outline China's specific requirements, which could include unique aspects due to national standards or practices.

### Assessing Compliance Risk

The most restrictive standard, as per ICAO rules, typically takes precedence to ensure the highest level of safety. For instance, if EASA CS-25 has more stringent requirements for a particular aspect of aircraft design than FAA Part 25, the EASA standard would be considered the benchmark for compliance in cross-registry transfers involving European jurisdictions.

### Providing Actionable Guidance

For cross-registry transfers, the following steps are recommended:
1. **Identify Applicable Regulations**: Determine which jurisdictions' regulations apply based on the current and intended registries of the aircraft.
2. **Conduct a Gap Analysis**: Compare the regulatory requirements of the current registry with those of the intended registry to identify any differences or gaps.
3. **Plan Modifications**: Based on the gap analysis, plan any necessary modifications to the aircraft to comply with the new registry's regulations.
4. **Obtain Approvals**: Secure approvals from the relevant aviation authorities for any modifications and for the transfer of the aircraft to the new registry.
5. **Maintain Compliance**: Ensure ongoing compliance with the regulatory requirements of the new registry, including any recurring inspections or audits.

In conclusion, when dealing with multi-jurisdictional aviation regulations, it's crucial to meticulously compare requirements, identify the most restrictive standards, and plan accordingly to ensure seamless compliance and registry transfers. Always consult the latest versions of regulatory documents and seek professional advice when navigating complex regulatory landscapes.


QUERY 2: Which jurisdiction has the most restrictive N1 vibration limits?



Given the absence of specific regulatory context and a compliance query in your request, I will provide a general framework for analyzing multi-jurisdictional regulations as per your instructions. This framework will outline how to approach identifying exact regulatory requirements, highlighting differences, citing specific sections, assessing compliance risk, and providing actionable guidance for cross-registry transfers.

### 1. Identify Exact Regulatory Requirements

To identify exact regulatory requirements, one must consult the relevant documents from the Federal Aviation Administration (FAA), European Union Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). For example, if the query pertains to aircraft design and airworthiness:

- **FAA**: Refer to the Federal Aviation Regulations (FARs), specifically parts that apply to the query, such as Part 25 for transport category airplanes.
- **EASA**: Consult the Certification Specifications (CS) for the relevant aircraft category, such as CS-25 for large aeroplanes.
- **CAAC**: Examine the Chinese civil aviation regulations and standards that correspond to the query.

### 2. Highlight Differences

Differences between jurisdictions can include various aspects such as agent types, timing, thresholds, and specific requirements for documentation, inspection, and testing. For instance:

- **Agent Types**: Different jurisdictions might recognize different types of agents (e.g., authorized representatives) with varying responsibilities and requirements.
- **Timing and Thresholds**: The timing for compliance and the thresholds for what requires compliance can differ significantly between FAA, EASA, and CAAC regulations.

### 3. Cite Specific Sections

When analyzing regulations, it's crucial to cite specific sections to ensure clarity and precision. For example:

- **FAA Part 25.851** might be cited for requirements related to the fire protection of aircraft.
- **EASA CS-25** requires compliance with specific standards for large aeroplanes, including those related to fire safety.

### 4. Assess Compliance Risk

Assessing compliance risk involves identifying the most restrictive standard among the jurisdictions, as per ICAO rules, to ensure that compliance with one set of regulations does not inadvertently lead to non-compliance with another. This requires a detailed comparison of the requirements from each jurisdiction.

### 5. Provide Actionable Guidance

For cross-registry transfers, actionable guidance would include:

- **Required Modifications**: Identifying any necessary modifications to the aircraft to meet the regulatory requirements of the new registry.
- **Documentation**: Ensuring all documentation, including certificates of airworthiness, is updated and compliant with the new jurisdiction's regulations.
- **Inspections and Testing**: Conducting any required inspections or tests to validate compliance with the new regulatory framework.

### Example Analysis

Given a query about fire protection requirements for commercial aircraft:

- **FAA Requirement**: Per FAA Part 25.851, certain fire protection standards must be met.
- **EASA Requirement**: EASA CS-25 requires compliance with specific fire safety standards.
- **CAAC Requirement**: The CAAC has its own set of regulations regarding fire protection, which might differ in specifics from FAA and EASA.

**Most Restrictive Standard**: After comparing, if EASA CS-25 imposes more stringent fire safety requirements than both FAA Part 25.851 and CAAC regulations, then EASA CS-25 would be considered the most restrictive standard for compliance purposes.

**Actionable Guidance**: For a cross-registry transfer involving an aircraft subject to these regulations, ensure that the aircraft meets the EASA CS-25 fire safety standards, undergo necessary modifications, and obtain updated certifications to comply with the most restrictive standard.

This approach ensures that regulatory requirements are met across different jurisdictions, minimizing compliance risk and facilitating smooth cross-registry transfers.


QUERY 3: What fire suppression agent is required under EASA regulations?



Given the absence of specific regulatory context and a compliance query in your request, I will provide a general framework for analyzing multi-jurisdictional regulations in aviation, focusing on the Federal Aviation Administration (FAA), European Union Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). This framework will outline how to identify exact regulatory requirements, highlight differences, cite specific sections, assess compliance risk, and provide actionable guidance for cross-registry transfers.

### 1. Identify Exact Regulatory Requirements

- **FAA Requirements**: For aircraft design and airworthiness, the FAA often references parts of the Federal Aviation Regulations (FARs), such as Part 25 for transport category airplanes. For example, fire protection requirements can be found in "FAA Part 25.851" which outlines the standards for the protection of the airplane against fire.
  
- **EASA Requirements**: EASA has its own set of regulations, with the Certification Specifications (CS) being crucial for aircraft design. For instance, "EASA CS-25" provides the airworthiness standards for large commercial airplanes, including detailed requirements for fire protection.

- **CAAC Requirements**: The CAAC has its regulations, which may align closely with either the FAA or EASA standards but can also have unique requirements. For example, CAAC might have specific regulations regarding the materials used in aircraft construction to meet local environmental standards.

### 2. Highlight Differences

- **Agent Types and Approval**: Differences can exist in the types of agents (e.g., fire-resistant materials) approved by each authority and the process for their approval. The FAA might approve certain materials under specific conditions (e.g., "FAA AC 25.851-1"), while EASA could have a different set of approved materials under "EASA CM-CS-004."

- **Timing and Thresholds**: The timing for compliance and the thresholds for what requires compliance can vary. For example, the FAA might require compliance with a new standard within 12 months for all new aircraft designs, while EASA could mandate compliance within 18 months but only for aircraft above a certain maximum takeoff mass.

### 3. Cite Specific Sections

- When referencing fire protection standards, one would cite "per FAA Part 25.851" for the FAA's requirements, "EASA CS-25.851" for EASA's, and the relevant section from CAAC's regulations for China's standards.

### 4. Assess Compliance Risk

- The most restrictive standard, as per ICAO rules, would typically be the one that an operator or manufacturer must adhere to, especially in cross-registry transfers. For instance, if EASA has the most stringent fire protection requirements, these would be the standards to follow for compliance, even if the aircraft is primarily operated under an FAA registry but occasionally flies into EASA jurisdiction.

### 5. Provide Actionable Guidance

- **For Cross-Registry Transfers**: Ensure that the aircraft meets the most restrictive standards of the jurisdictions it will operate in. This might involve modifying the aircraft to meet EASA's fire protection standards if it's being transferred from an FAA registry to one that falls under EASA jurisdiction.
- **Documentation**: Maintain detailed documentation of compliance with all relevant regulations, including certificates of airworthiness, modification records, and approval documents from the respective aviation authorities.
- **Ongoing Compliance**: Regularly review regulatory updates from the FAA, EASA, and CAAC to ensure ongoing compliance, especially as standards evolve.

In conclusion, without specific context or a query, the approach to analyzing and complying with multi-jurisdictional aviation regulations involves identifying the exact requirements from each relevant authority, highlighting differences, citing specific regulatory sections, assessing compliance risk by identifying the most restrictive standard, and providing actionable guidance for ensuring compliance, especially in scenarios involving cross-registry transfers.


QUERY 4: What inspection intervals apply to fire suppression systems?



Given the absence of specific regulatory context and a compliance query in your request, I will provide a general framework for analyzing multi-jurisdictional regulations in aviation, focusing on the Federal Aviation Administration (FAA), European Aviation Safety Agency (EASA), and Civil Aviation Administration of China (CAAC). This framework will outline how to identify exact regulatory requirements, highlight differences, cite specific sections, assess compliance risk, and provide actionable guidance for cross-registry transfers.

### 1. Identify Exact Regulatory Requirements

- **FAA Requirements**: For aircraft design and airworthiness, the FAA often references Part 25 of the Federal Aviation Regulations (FARs), which pertains to transport category airplanes. For example, fire protection requirements can be found in Part 25.851.
- **EASA Requirements**: EASA's Certification Specifications for Large Aeroplanes (CS-25) mirror many of the FAA's Part 25 requirements but may have additional or differing standards. EASA CS-25 requires compliance with specific airworthiness and environmental standards.
- **CAAC Requirements**: The CAAC has its own set of regulations, which may align closely with either the FAA or EASA standards, but can also include unique requirements based on Chinese aviation law and regulations.

### 2. Highlight Differences

- **Agent Types and Responsibilities**: Differences may exist in the roles and responsibilities of authorized representatives or agents between jurisdictions. For instance, the FAA might require a specific type of agent for certain transactions, whereas EASA or CAAC might have different or additional requirements.
- **Timing and Thresholds**: Regulatory bodies may have different timing requirements for inspections, maintenance, or reporting. Thresholds for what constitutes a major modification or when certain regulations apply can also vary.

### 3. Cite Specific Sections

- **FAA Part 25.851** deals with fire protection requirements for transport category airplanes.
- **EASA CS-25** requires that large aeroplanes meet specific standards for airworthiness, including those related to fire protection, which can be found in the relevant subsections of CS-25.
- **CAAC Regulations**, while less detailed in publicly available English translations, would reference specific sections of Chinese aviation law and implementing regulations.

### 4. Assess Compliance Risk

The most restrictive standard, as per ICAO rules, would typically be the one that an operator or manufacturer must adhere to, especially in cases of cross-registry transfers or international operations. For example, if the EASA has a more stringent requirement for a particular aspect of aircraft maintenance than the FAA, an operator seeking to register an aircraft in Europe after previously being registered in the United States would need to comply with the EASA's standards.

### 5. Provide Actionable Guidance

For cross-registry transfers, the following steps are recommended:
- **Conduct a Gap Analysis**: Identify all regulatory differences between the current and intended registry jurisdictions.
- **Modify to Comply**: Implement any necessary modifications to the aircraft to meet the more stringent standards of the new registry.
- **Documentation and Approval**: Ensure all documentation, including airworthiness certificates and maintenance records, is updated and approved by the relevant authorities.
- **Training and Operations**: Update operational procedures and provide necessary training to crew members to comply with the new regulatory requirements.

In conclusion, without specific context or a query, this analysis provides a general approach to navigating multi-jurisdictional aviation regulations. For precise guidance, the exact regulatory requirements, differences between jurisdictions, and specific sections of the regulations in question must be carefully examined.

In [20]:
# =====================================================
# CELL 18: Export and Save Results
# =====================================================
"""
Save your analysis results for documentation or reporting.
"""

import json
from datetime import datetime

# Create analysis report
report = {
    "timestamp": datetime.now().isoformat(),
    "configuration": {
        "embedding_model": config.embedding_model,
        "reranker_model": config.reranker_model,
        "llm_model": config.llm_model,
        "num_documents": len(documents),
        "num_chunks": len(chunks)
    },
    "queries": results
}

# Save as JSON
report_json = json.dumps(report, indent=2)
with open('/content/realm_compliance_report.json', 'w') as f:
    f.write(report_json)

print("✅ Report saved to: /content/realm_compliance_report.json")
print("\n📄 Report Summary:")
print(f"   Total Queries: {len(results)}")
print(f"   Documents Indexed: {len(documents)}")
print(f"   Chunks Created: {len(chunks)}")

✅ Report saved to: /content/realm_compliance_report.json

📄 Report Summary:
   Total Queries: 4
   Documents Indexed: 1095
   Chunks Created: 1041


In [21]:
# =====================================================
# CELL 19: Advanced - Salient Span Masking Demo
# =====================================================
"""
OPTIONAL: Demonstration of REALM's Salient Span Masking pre-training.
This is conceptual - full REALM pre-training requires significant compute.

In production, you would:
1. Mask important regulatory terms
2. Train retriever to find documents that help unmask
3. This teaches the retriever domain expertise
"""

print("🧪 REALM SALIENT SPAN MASKING DEMO")
print("="*80)

example_text = """The [MASK] must be inspected every 500 flight hours according to EASA Part-145."""

print("\nOriginal (Masked) Text:")
print(f"  {example_text}")

print("\nREALM Training Objective:")
print("  The retriever must learn to retrieve the document that contains")
print("  'fire suppression bottle' to fill the [MASK].")

print("\n💡 Why This Matters:")
print("  - Forces retriever to understand domain-specific relationships")
print("  - Unlike standard embeddings, it learns that '500 flight hours'")
print("    is semantically linked to specific maintenance procedures")
print("  - This is why REALM excels at technical/regulatory retrieval")

# =====================================================
# CELL 20: System Summary and Next Steps
# =====================================================
"""
Summary of the REALM RAG pipeline and recommendations.
"""

print("\n" + "="*80)
print("🎯 REALM RAG SYSTEM SUMMARY")
print("="*80)

print("""
✅ **What We Built:**
   - REALM-inspired RAG pipeline for aviation compliance
   - Two-stage retrieval: Dense (MIPS) + Cross-encoder reranking
   - Multi-jurisdictional analysis (FAA, EASA, CAAC)
   - Probabilistic document scoring P(z|x)

📊 **Key Advantages Over Standard RAG:**
   1. Learned Relevance: Reranker understands domain relationships
   2. Better handling of technical terminology (e.g., "N1 vibration")
   3. Cross-document reasoning for regulatory deltas
   4. Confidence scoring for retrieved documents

🚀 **Production Enhancements:**
   1. Add real compliance documents (PDFs from regulatory bodies)
   2. Implement full REALM pre-training with salient span masking
   3. Add citation tracking (show exact regulation sections)
   4. Deploy with FastAPI for web service
   5. Add conversation memory for follow-up questions
   6. Implement regulatory change monitoring (alert on updates)

📚 **To Add Your Own Documents:**
   - Replace synthetic_docs in CELL 4 with PyPDFLoader
   - Upload PDFs to Colab: Files → Upload
   - Load with: PyPDFLoader("/content/your_file.pdf").load()

🔧 **To Improve Retrieval:**
   - Increase top_k_retrieval for broader search
   - Use domain-specific embedding models (legal-bert, etc.)
   - Fine-tune reranker on aviation compliance Q&A pairs
""")

print("\n" + "="*80)
print("✨ REALM RAG Pipeline Ready for Queries! ✨")
print("="*80)

print("\nUse: query_realm_system('Your question here') to test the system")

🧪 REALM SALIENT SPAN MASKING DEMO

Original (Masked) Text:
  The [MASK] must be inspected every 500 flight hours according to EASA Part-145.

REALM Training Objective:
  The retriever must learn to retrieve the document that contains
  'fire suppression bottle' to fill the [MASK].

💡 Why This Matters:
  - Forces retriever to understand domain-specific relationships
  - Unlike standard embeddings, it learns that '500 flight hours'
    is semantically linked to specific maintenance procedures
  - This is why REALM excels at technical/regulatory retrieval

🎯 REALM RAG SYSTEM SUMMARY

✅ **What We Built:**
   - REALM-inspired RAG pipeline for aviation compliance
   - Two-stage retrieval: Dense (MIPS) + Cross-encoder reranking
   - Multi-jurisdictional analysis (FAA, EASA, CAAC)
   - Probabilistic document scoring P(z|x)

📊 **Key Advantages Over Standard RAG:**
   1. Learned Relevance: Reranker understands domain relationships
   2. Better handling of technical terminology (e.g., "N1 vibrati

In [1]:
!pip install -q langchain==0.3.0 langchain-community langchain-core langchain-groq sentence-transformers faiss-cpu pypdf groq spacy torch
# !python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [2]:
import os
import torch
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from datasets import Dataset
import spacy
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers.cross_encoder import CrossEncoder
from IPython.display import display, Markdown

ModuleNotFoundError: No module named 'numpy.strings'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/AdvanceRAG Problems/REALM Data"
%pwd

In [ ]:
!ls

Load and Split PDFs
Assume user uploads PDFs for the knowledge corpus, e.g., far.pdf, easa.pdf, caac.pdf, etc.
For demonstration, use a placeholder PDF or adjust paths accordingly.

In [ ]:
pdf_paths = ["CAAC_REALM_RAG.pdf", "FAA_REALM_RAG.pdf", "MSG3_REALM_RAG.pdf", "NTSB_REALM_RAG.pdf"]  # Replace with actual paths

documents = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    documents.extend(docs)

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)
print(f"Total Chunks Created: {len(chunks)}")

Simulate REALM Pre-training with Salient Span Masking
We use spaCy for NER to identify salient spans (entities), mask them, and create training pairs for fine-tuning the retriever.

In [ ]:
nlp = spacy.load("en_core_web_sm")

def create_masked_pairs(chunks, mask_prob=0.15, min_spans=1):
    anchors = []
    positives = []
    for chunk in tqdm(chunks):
        text = chunk.page_content
        doc = nlp(text)
        entities = [(ent.text, ent.start_char, ent.end_char) for ent in doc.ents if ent.label_ in ["ORG", "GPE", "LAW", "NORP", "PRODUCT"]]
        if len(entities) < min_spans:
            continue
        # Randomly select spans to mask
        num_masks = max(1, int(len(entities) * mask_prob))
        mask_indices = np.random.choice(len(entities), num_masks, replace=False)
        masked_text = text
        offset = 0
        for idx in sorted(mask_indices):
            ent_text, start, end = entities[idx]
            masked_text = masked_text[:start - offset] + "[MASK]" + masked_text[end - offset:]
            offset += (end - start) - len("[MASK]")
        anchors.append(masked_text)
        positives.append(text)
    return anchors, positives

anchors, positives = create_masked_pairs(chunks)

# Create dataset for fine-tuning
train_anchors, eval_anchors, train_positives, eval_positives = train_test_split(anchors, positives, test_size=0.1)
train_examples = [{"anchor": a, "positive": p} for a, p in zip(train_anchors, train_positives)]
eval_examples = [{"anchor": a, "positive": p} for a, p in zip(eval_anchors, eval_positives)]

train_dataset = Dataset.from_list(train_examples)
eval_dataset = Dataset.from_list(eval_examples)

Fine-Tune Sentence Transformer for Domain-Specific Retriever

In [ ]:
os.environ["WANDB_DISABLED"] = "true"


In [11]:
model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

loss = MultipleNegativesRankingLoss(model)

args = SentenceTransformerTrainingArguments(
    output_dir="finetuned_retriever",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler="no_duplicates",
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
)

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained("finetuned_aviation_retriever")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss
100,0.004600,0.000003


Load Fine-Tuned Embeddings and Create Vectorstore

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="finetuned_aviation_retriever")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

/tmp/ipython-input-3465819522.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="finetuned_aviation_retriever")


Define LLM

In [13]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')  # Replace with your Groq API key

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=groq_api_key
)

Define Prompt and Reranker

In [14]:
system_prompt = (
    "You are an expert aviation compliance auditor. "
    "Use the following context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Create Retrieval Chain with Memory
Using create_history_aware_retriever for conversational memory

In [27]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [20]:
rag_chain.output_schema

langchain_core.utils.pydantic.RunnableSequenceOutput

Function to Rerank Documents

In [28]:
def rerank_docs(question, docs):
    pairs = [[question, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    scored_docs = list(zip(docs, scores))
    sorted_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in sorted_docs]

Example Query and Response (Before and After Reranking)
Maintain chat history for memory

In [29]:
chat_history = []

question = "We are moving an aircraft from a US registry to a French registry. What specific modifications are required for the fire suppression system under EASA CS-25 that differ from FAA Part 25?"

# Retrieve docs
retrieved_docs = retriever.invoke(question)

# Show pre-reranked
print("\n🔹 Top Retrieved Chunks (Before Reranking):")
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Chunk {i+1} ---")
    print(f"Page: {doc.metadata.get('page', 'Unknown')}")
    print(f"Content:\n{doc.page_content[:300]}...")


🔹 Top Retrieved Chunks (Before Reranking):

--- Chunk 1 ---
Page: 455
Content:
AIM 4/20/23 
FIG 5−4−33 
EFVS Operation to 100 ft Above the TDZE 
d. EFVS Equipment Requirements. An EFVS that is installed on a U.S.−registered aircraft and is used to 
conduct EFVS operations must conform to an FAA−type design approval (i.e., a type certificate (TC), amended 
TC, or supplemental t...

--- Chunk 2 ---
Page: 18
Content:
U.S. Department 
of Transportation 
Federal Aviation 
Administration 
AERONAUTICAL 
INFORMATION 
MANUAL 
Change 2 
March 21, 2024 
DO NOT DESTROY 
BASIC DATED 
April 20, 2023...

--- Chunk 3 ---
Page: 734
Content:
4/20/23 AIM 
Appendix 3. Abbreviations/Acronyms 
As used in this manual, the following abbreviations/acronyms have the meanings indicated. 
Abbreviation/ 
Acronym 
Meaning 
A A W U ..... Alaskan Aviation Weather Unit 
A A S ....... Airport Advisory Service 
A A M ...... Advanced Air Mobility 
A C .....

--- Chunk 4 ---
Page: 282
Content:
4/20/23 AIM 
may be necessa

In [30]:
# Rerank
reranked_docs = rerank_docs(question, retrieved_docs)

# Show reranked
print("\n🔸 Reranked Chunks:")
for i, doc in enumerate(reranked_docs):
    print(f"\n--- Reranked Chunk {i+1} ---")
    print(f"Page: {doc.metadata.get('page', 'Unknown')}")
    print(f"Content:\n{doc.page_content[:300]}...")


🔸 Reranked Chunks:

--- Reranked Chunk 1 ---
Page: 455
Content:
AIM 4/20/23 
FIG 5−4−33 
EFVS Operation to 100 ft Above the TDZE 
d. EFVS Equipment Requirements. An EFVS that is installed on a U.S.−registered aircraft and is used to 
conduct EFVS operations must conform to an FAA−type design approval (i.e., a type certificate (TC), amended 
TC, or supplemental t...

--- Reranked Chunk 2 ---
Page: 282
Content:
4/20/23 AIM 
may be necessary for ATC to issue further speed adjustments after approach clearance is issued to maintain 
separation between successive arrivals. Under such circumstances, previously issued speed adjustments will be 
restated if that speed is to be maintained or additional speed adjus...

--- Reranked Chunk 3 ---
Page: 18
Content:
U.S. Department 
of Transportation 
Federal Aviation 
Administration 
AERONAUTICAL 
INFORMATION 
MANUAL 
Change 2 
March 21, 2024 
DO NOT DESTROY 
BASIC DATED 
April 20, 2023...

--- Reranked Chunk 4 ---
Page: 734
Content:
4/20/23 AIM 
Ap

In [26]:
# Update history
chat_history.extend([{"role": "human", "content": question}, {"role": "ai", "content": result_before["answer"]}])

# Answer after reranking
# To use reranked, we can override context
context_after = "\n\n".join([doc.page_content for doc in reranked_docs[:3]])
result_after = question_answer_chain.invoke({"input": question, "chat_history": chat_history, "context": context_after})
display(Markdown("### Final Answer (After Reranking):"))
display(Markdown(result_after))

NameError: name 'result_before' is not defined

In [ ]:
# =====================================================
# CELL 1: Install Dependencies
# =====================================================
"""
REALM RAG Pipeline for Autonomous Regulatory Compliance Auditor
----------------------------------------------------------------
This notebook implements a REALM-inspired RAG system for cross-jurisdictional
aviation safety compliance. Unlike standard RAG, REALM pre-trains the retriever
to understand domain-specific relationships between queries and documents.

Key Differences from Standard RAG:
1. Learned Relevance: Retriever is fine-tuned on domain-specific data
2. Probabilistic Retrieval: Uses P(z|x) to rank documents
3. Salient Span Masking: Pre-training forces retriever expertise
"""

!pip install -q langchain==0.3.0 langchain-community langchain-core langchain-groq
!pip install -q sentence-transformers faiss-cpu pypdf groq
!pip install -q transformers torch datasets
!pip install -q scikit-learn pandas

# =====================================================
# CELL 2: Core Imports
# =====================================================
import os
import torch
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from IPython.display import display, Markdown

# LangChain 0.3 imports
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_groq import ChatGroq

# Transformer models
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.cross_encoder import CrossEncoder
from transformers import AutoTokenizer, AutoModel

print("✅ All dependencies imported successfully!")

# =====================================================
# CELL 3: Configuration and Helper Classes
# =====================================================
@dataclass
class REALMConfig:
    """Configuration for REALM RAG pipeline"""
    chunk_size: int = 800
    chunk_overlap: int = 100
    top_k_retrieval: int = 10
    top_k_rerank: int = 5
    embedding_model: str = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
    reranker_model: str = "cross-encoder/ms-marco-MiniLM-L6-v2"
    llm_model: str = "llama-3.3-70b-versatile"
    temperature: float = 0.1

config = REALMConfig()

class REALMRetriever:
    """
    REALM-inspired retriever that implements probabilistic document selection.

    Key Features:
    - Dual-stage retrieval (dense + reranking)
    - Confidence scoring using P(z|x)
    - Domain-specific relevance learning
    """

    def __init__(self, vectorstore, reranker_model: str):
        self.vectorstore = vectorstore
        self.reranker = CrossEncoder(reranker_model)

    def retrieve_and_score(self, query: str, top_k: int = 10) -> List[Tuple]:
        """
        Retrieve documents with REALM-style probabilistic scoring.
        Returns: List of (document, relevance_score) tuples
        """
        # Stage 1: Dense retrieval using MIPS (Maximum Inner Product Search)
        initial_docs = self.vectorstore.similarity_search(query, k=top_k)

        # Stage 2: Rerank using cross-encoder (simulates P(z|x))
        pairs = [[query, doc.page_content] for doc in initial_docs]
        scores = self.reranker.predict(pairs)

        # Normalize scores to probabilities (softmax)
        exp_scores = np.exp(scores - np.max(scores))
        probabilities = exp_scores / exp_scores.sum()

        # Combine documents with their P(z|x) scores
        scored_docs = list(zip(initial_docs, probabilities))
        sorted_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)

        return sorted_docs

    def get_top_documents(self, query: str, top_k: int = 5):
        """Get top-k documents after reranking"""
        scored_docs = self.retrieve_and_score(query, top_k=config.top_k_retrieval)
        return [doc for doc, _ in scored_docs[:top_k]]

print("✅ Configuration and helper classes defined!")

# =====================================================
# CELL 4: Document Loading and Preprocessing
# =====================================================
"""
For this demo, we'll simulate aviation compliance documents.
In production, you would load actual FAA, EASA, and CAAC regulations.

Upload your PDF files to Colab, or use the sample code below.
"""

# Example: Load a single PDF (replace with your compliance documents)
# Uncomment and modify the path below:
# pdf_path = "/content/your_compliance_document.pdf"
# loader = PyPDFLoader(pdf_path)
# documents = loader.load()

# For demonstration, create synthetic compliance documents
from langchain_core.documents import Document

synthetic_docs = [
    Document(
        page_content="""
        FAA Part 25 Section 851: Fire Suppression Systems

        All transport category aircraft must be equipped with a fire extinguishing
        system for the engine compartment. The system must use Halon 1301 or an
        approved alternative agent. Discharge must occur within 5 seconds of
        activation. N1 vibration limits during fire suppression activation must
        not exceed 3.0 units to prevent compressor damage.

        Inspection Requirements: Visual inspection every 500 flight hours.
        Hydrostatic testing of cylinders every 5 years per DOT-E-8876 specification.
        """,
        metadata={"source": "FAA_Part25", "section": "851", "jurisdiction": "USA"}
    ),
    Document(
        page_content="""
        EASA CS-25 Section 851: Fire Protection Systems

        European regulations require fire suppression systems to meet additional
        environmental standards. Unlike FAA regulations, EASA mandates the use of
        HFC-227ea (rather than Halon 1301) due to ozone depletion concerns under
        Montreal Protocol compliance.

        Registry Transfer Requirements: Aircraft moving from FAA to EASA registry
        must retrofit fire suppression systems with approved HFC agents. Activation
        time must be within 3 seconds (more stringent than FAA's 5 seconds).

        Inspection: Every 400 flight hours per Part-145 maintenance organization.
        """,
        metadata={"source": "EASA_CS25", "section": "851", "jurisdiction": "EU"}
    ),
    Document(
        page_content="""
        CAAC CCAR-25 Section 851: Fire Extinguishing Requirements

        Chinese aviation regulations align with FAA Part 25 but include additional
        requirements for domestic operations. Fire suppression agent must be approved
        by both CAAC and ICAO standards.

        N1 Vibration Monitoring: During fire suppression system discharge, continuous
        monitoring of N1 vibration is required with automatic engine shutdown if
        vibration exceeds 2.5 units (more conservative than FAA's 3.0 limit).

        For aircraft registered in multiple jurisdictions, the most restrictive
        standard applies per ICAO Annex 8 guidelines.
        """,
        metadata={"source": "CAAC_CCAR25", "section": "851", "jurisdiction": "China"}
    ),
    Document(
        page_content="""
        MSG-3 Maintenance Steering Group: Turbine Engine Fire Protection

        Maintenance Task 71-31-01: Fire Suppression Bottle Inspection

        Interval: 500 FH (Flight Hours) or 12 months, whichever occurs first
        Task Type: General Visual Inspection (GVI)

        Cross-Reference:
        - FAA AD 2023-08-15 (Airworthiness Directive on bottle pressure)
        - EASA AD 2023-0089 (Equivalent European directive)

        Historical Note: After incident AA-2019-447, FAA reduced N1 vibration
        tolerance from 3.5 to 3.0 units during suppression activation.
        """,
        metadata={"source": "MSG3_Logs", "task": "71-31-01", "type": "maintenance"}
    ),
    Document(
        page_content="""
        NTSB Accident Report AAR-19/03: Engine Fire Suppression Failure

        Incident Date: March 15, 2019
        Aircraft: Boeing 737-800, N-Registration (FAA)

        Probable Cause: Fire suppression bottle failed to discharge due to
        expired hydrostatic test certification. Maintenance logs showed last
        inspection at 1,200 flight hours (exceeded 500 FH requirement).

        Safety Recommendation: NTSB recommends mandatory electronic tracking
        of fire suppression system inspections tied to aircraft digital logbooks.

        Regulatory Impact: Led to FAA AD 2023-08-15 requiring reduced inspection
        intervals for high-utilization fleets.
        """,
        metadata={"source": "NTSB", "report": "AAR-19/03", "type": "accident_analysis"}
    )
]

# Use synthetic docs for demo (replace with your real document loading)
documents = synthetic_docs

print(f"✅ Loaded {len(documents)} compliance documents")
for doc in documents:
    print(f"  - {doc.metadata.get('source', 'Unknown')}: {len(doc.page_content)} chars")

# =====================================================
# CELL 5: Text Chunking with Overlap
# =====================================================
"""
REALM requires careful chunking to preserve regulatory context.
We use RecursiveCharacterTextSplitter to maintain semantic coherence.
"""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=config.chunk_size,
    chunk_overlap=config.chunk_overlap,
    separators=["\n\n", "\n", ". ", " ", ""],
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

# Add chunk metadata
for i, chunk in enumerate(chunks):
    chunk.metadata["chunk_id"] = i

print(f"✅ Created {len(chunks)} chunks from {len(documents)} documents")
print(f"   Chunk size: {config.chunk_size} chars | Overlap: {config.chunk_overlap} chars")

# Preview first chunk
print("\n📄 Sample Chunk:")
print(f"Source: {chunks[0].metadata.get('source', 'N/A')}")
print(f"Content: {chunks[0].page_content[:300]}...")

# =====================================================
# CELL 6: Initialize Embeddings (REALM-style Dense Retrieval)
# =====================================================
"""
REALM uses a BERT-based encoder for dense retrieval.
We use multi-qa-mpnet which is optimized for question-answering retrieval.

Key: This model has been pre-trained on MS MARCO and Natural Questions,
simulating REALM's Wikipedia-scale pre-training.
"""

embedding_model = HuggingFaceEmbeddings(
    model_name=config.embedding_model,
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
    encode_kwargs={'normalize_embeddings': True}  # For cosine similarity
)

print(f"✅ Initialized embeddings: {config.embedding_model}")
print(f"   Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# Test embedding
test_embedding = embedding_model.embed_query("What are the fire suppression requirements?")
print(f"   Embedding dimension: {len(test_embedding)}")

# =====================================================
# CELL 7: Create FAISS Vector Store (MIPS Index)
# =====================================================
"""
FAISS implements Maximum Inner Product Search (MIPS), which is core to REALM.
This allows efficient retrieval from large knowledge corpora.
"""

vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embedding_model
)

print(f"✅ FAISS vector store created with {len(chunks)} vectors")
print(f"   Index type: MIPS (Maximum Inner Product Search)")

# Test similarity search
test_query = "N1 vibration limits"
test_results = vectorstore.similarity_search(test_query, k=3)
print(f"\n🔍 Test Query: '{test_query}'")
print(f"   Retrieved {len(test_results)} documents")

# =====================================================
# CELL 8: Initialize REALM Retriever
# =====================================================
"""
The REALMRetriever implements the two-stage retrieval process:
1. Dense retrieval (MIPS) - fast, recalls top-k candidates
2. Cross-encoder reranking - slow but accurate, simulates P(z|x)
"""

realm_retriever = REALMRetriever(
    vectorstore=vectorstore,
    reranker_model=config.reranker_model
)

print(f"✅ REALM Retriever initialized")
print(f"   Stage 1: Dense retrieval (top-{config.top_k_retrieval})")
print(f"   Stage 2: Cross-encoder reranking (top-{config.top_k_rerank})")

# Test retrieval with scoring
test_query = "What modifications are required when transferring from FAA to EASA registry?"
scored_results = realm_retriever.retrieve_and_score(test_query, top_k=5)

print(f"\n🎯 Test Retrieval with P(z|x) Scores:")
print(f"Query: {test_query}\n")
for i, (doc, prob) in enumerate(scored_results[:3], 1):
    print(f"Rank {i} | P(z|x) = {prob:.4f}")
    print(f"Source: {doc.metadata.get('source', 'N/A')}")
    print(f"Content: {doc.page_content[:200]}...\n")

# =====================================================
# CELL 9: Initialize LLM with Groq
# =====================================================
"""
Configure the LLM for generating compliance answers.
Low temperature ensures factual, deterministic responses.
"""

from google.colab import userdata

llm = ChatGroq(
    model=config.llm_model,
    temperature=config.temperature,
    api_key=userdata.get('GROQ_API_KEY')  # Set in Colab Secrets
)

print(f"✅ LLM initialized: {config.llm_model}")
print(f"   Temperature: {config.temperature} (deterministic)")

# Test LLM
test_response = llm.invoke([HumanMessage(content="Say 'LLM ready' if you can read this.")])
print(f"   Status: {test_response.content}")

# =====================================================
# CELL 10: Create REALM-Style Prompt Template
# =====================================================
"""
REALM prompt emphasizes:
1. Multi-document reasoning (cross-jurisdictional analysis)
2. Explicit citation of source regulations
3. Highlighting contradictions between jurisdictions
"""

realm_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""You are an expert aviation compliance auditor analyzing multi-jurisdictional regulations.

Your task is to:
1. **Identify Exact Regulatory Requirements** from FAA, EASA, and CAAC documents
2. **Highlight Differences** between jurisdictions (e.g., different agent types, timing, thresholds)
3. **Cite Specific Sections** (e.g., "per FAA Part 25.851" or "EASA CS-25 requires...")
4. **Assess Compliance Risk** - which regulation is most restrictive?
5. **Provide Actionable Guidance** for cross-registry transfers

**Critical**: Base your answer ONLY on the provided context. If regulations conflict, state both requirements clearly."""),

    HumanMessage(content="""**Retrieved Regulatory Context:**

{context}

---

**Compliance Query:**
{question}

**Analysis Instructions:**
- Compare FAA, EASA, and CAAC requirements if all are present
- Identify the most restrictive standard (this applies per ICAO rules)
- Note any required modifications for registry transfers
- Cite specific sections and standards

**Your Expert Analysis:**""")
])

print("✅ REALM prompt template created")
print("   Focus: Multi-jurisdictional regulatory analysis")

# =====================================================
# CELL 11: Build REALM RAG Chain (LangChain 0.3 Runnables)
# =====================================================
"""
LangChain 0.3 uses LCEL (LangChain Expression Language) with Runnables.
This chain implements the REALM pipeline: Retrieve → Rerank → Generate
"""

def format_docs(docs):
    """Format retrieved documents with metadata for context"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        section = doc.metadata.get('section', '')
        jurisdiction = doc.metadata.get('jurisdiction', '')

        header = f"[Document {i}: {source}"
        if section:
            header += f" §{section}"
        if jurisdiction:
            header += f" ({jurisdiction})"
        header += "]"

        formatted.append(f"{header}\n{doc.page_content}\n")

    return "\n---\n".join(formatted)

# Custom retriever function that uses REALM retriever
def realm_retrieve(query: str):
    """REALM-style retrieval with probabilistic scoring"""
    return realm_retriever.get_top_documents(query, top_k=config.top_k_rerank)

# Build the REALM RAG chain using LCEL
realm_rag_chain = (
    {
        "context": lambda x: format_docs(realm_retrieve(x["question"])),
        "question": lambda x: x["question"]
    }
    | realm_prompt
    | llm
    | StrOutputParser()
)

print("✅ REALM RAG Chain constructed using LangChain 0.3 Runnables")
print("   Pipeline: Query → MIPS Retrieval → Cross-Encoder Rerank → LLM Generation")

# =====================================================
# CELL 12: Test Query 1 - Cross-Jurisdictional Analysis
# =====================================================
"""
Test Case 1: The exact scenario from your requirements.
This tests REALM's ability to identify delta between jurisdictions.
"""

query_1 = """We are moving an aircraft from a US (FAA) registry to a French (EASA) registry.
What specific modifications are required for the fire suppression system under EASA CS-25
that differ from FAA Part 25?"""

print("🔍 QUERY 1: Cross-Jurisdictional Registry Transfer\n")
print(f"Question: {query_1}\n")
print("="*80)

# Show retrieved documents with scores
print("\n📚 REALM Retrieval Process:\n")
scored_docs = realm_retriever.retrieve_and_score(query_1, top_k=config.top_k_retrieval)
print(f"Stage 1: Retrieved {len(scored_docs)} candidate documents")
print(f"\nStage 2: Top-{config.top_k_rerank} after reranking (with P(z|x) scores):\n")

for i, (doc, prob) in enumerate(scored_docs[:config.top_k_rerank], 1):
    print(f"Rank {i} | P(z|x) = {prob:.4f} | Source: {doc.metadata.get('source', 'N/A')}")

# Generate answer
print("\n" + "="*80)
print("\n🤖 REALM RAG ANSWER:\n")

response_1 = realm_rag_chain.invoke({"question": query_1})
display(Markdown(response_1))

# =====================================================
# CELL 13: Test Query 2 - Technical Specification Lookup
# =====================================================
"""
Test Case 2: Testing REALM's learned relevance for technical terms.
"N1 Vibration Limit" should retrieve the right document even though
the query and document don't have high lexical overlap.
"""

query_2 = """What is the N1 vibration limit during fire suppression system activation,
and how does it differ between FAA and CAAC regulations?"""

print("\n" + "="*80)
print("\n🔍 QUERY 2: Technical Specification Comparison\n")
print(f"Question: {query_2}\n")
print("="*80)

# Show retrieval
print("\n📚 Retrieved Documents:\n")
scored_docs_2 = realm_retriever.retrieve_and_score(query_2, top_k=config.top_k_retrieval)

for i, (doc, prob) in enumerate(scored_docs_2[:3], 1):
    print(f"\nRank {i} | P(z|x) = {prob:.4f}")
    print(f"Source: {doc.metadata.get('source', 'N/A')}")
    print(f"Preview: {doc.page_content[:250]}...")

# Generate answer
print("\n" + "="*80)
print("\n🤖 REALM RAG ANSWER:\n")

response_2 = realm_rag_chain.invoke({"question": query_2})
display(Markdown(response_2))

# =====================================================
# CELL 14: Test Query 3 - Maintenance History Analysis
# =====================================================
"""
Test Case 3: Cross-referencing maintenance logs with accident reports.
Tests REALM's ability to connect historical data with current regulations.
"""

query_3 = """Based on NTSB accident reports, what changes were made to fire suppression
inspection intervals, and do current MSG-3 procedures reflect these changes?"""

print("\n" + "="*80)
print("\n🔍 QUERY 3: Regulatory Change History Analysis\n")
print(f"Question: {query_3}\n")
print("="*80)

response_3 = realm_rag_chain.invoke({"question": query_3})
display(Markdown(response_3))

# =====================================================
# CELL 15: Comparative Analysis - Standard RAG vs REALM
# =====================================================
"""
Compare standard RAG (no reranking) with REALM RAG (with reranking).
This demonstrates the value of the REALM approach.
"""

print("\n" + "="*80)
print("📊 COMPARATIVE ANALYSIS: Standard RAG vs REALM RAG")
print("="*80)

test_query = "Registry transfer fire suppression requirements"

# Standard RAG (just vector similarity)
print("\n1️⃣ STANDARD RAG (Vector Similarity Only):\n")
standard_docs = vectorstore.similarity_search(test_query, k=3)
for i, doc in enumerate(standard_docs, 1):
    print(f"Rank {i} | Source: {doc.metadata.get('source', 'N/A')}")

# REALM RAG (with reranking)
print("\n2️⃣ REALM RAG (With Cross-Encoder Reranking):\n")
realm_docs = realm_retriever.retrieve_and_score(test_query, top_k=5)
for i, (doc, prob) in enumerate(realm_docs[:3], 1):
    print(f"Rank {i} | P(z|x) = {prob:.4f} | Source: {doc.metadata.get('source', 'N/A')}")

print("\n💡 Key Insight: REALM reranking often changes document order significantly,")
print("   retrieving documents that are semantically relevant but lexically distant.")

# =====================================================
# CELL 16: Interactive Query Interface
# =====================================================
"""
Interactive interface for testing your own compliance queries.
"""

def query_realm_system(question: str, show_sources: bool = True):
    """
    Query the REALM RAG system with detailed output.

    Args:
        question: Compliance query
        show_sources: Whether to display source documents
    """
    print("🔍 QUERY:", question)
    print("="*80)

    if show_sources:
        print("\n📚 Retrieved Sources (with REALM scores):\n")
        scored_docs = realm_retriever.retrieve_and_score(question, top_k=config.top_k_retrieval)
        for i, (doc, prob) in enumerate(scored_docs[:config.top_k_rerank], 1):
            print(f"\n[{i}] P(z|x) = {prob:.4f} | {doc.metadata.get('source', 'N/A')}")
            print(f"    {doc.page_content[:200]}...")

    print("\n" + "="*80)
    print("🤖 ANSWER:\n")

    response = realm_rag_chain.invoke({"question": question})
    display(Markdown(response))

    return response

# Example usage:
print("✅ Interactive query function ready!")
print("\nExample usage:")
print("```python")
print('query_realm_system("Your compliance question here")')
print("```")

# Test it:
query_realm_system(
    "What are the hydrostatic testing requirements for fire suppression bottles?",
    show_sources=True
)

# =====================================================
# CELL 17: Batch Query Analysis
# =====================================================
"""
Run multiple queries in batch to analyze system performance.
"""

batch_queries = [
    "What is the maximum activation time for fire suppression systems?",
    "Which jurisdiction has the most restrictive N1 vibration limits?",
    "What fire suppression agent is required under EASA regulations?",
    "What inspection intervals apply to fire suppression systems?"
]

print("📋 BATCH QUERY ANALYSIS")
print("="*80)

results = []
for i, query in enumerate(batch_queries, 1):
    print(f"\n[Query {i}/{len(batch_queries)}] {query}")
    response = realm_rag_chain.invoke({"question": query})
    results.append({"query": query, "response": response})
    print(f"✅ Completed\n")

# Display all results
print("\n" + "="*80)
print("📊 ALL RESULTS:")
print("="*80)

for i, result in enumerate(results, 1):
    print(f"\n{'='*80}")
    print(f"QUERY {i}: {result['query']}")
    print(f"{'='*80}\n")
    display(Markdown(result['response']))

# =====================================================
# CELL 18: Export and Save Results
# =====================================================
"""
Save your analysis results for documentation or reporting.
"""

import json
from datetime import datetime

# Create analysis report
report = {
    "timestamp": datetime.now().isoformat(),
    "configuration": {
        "embedding_model": config.embedding_model,
        "reranker_model": config.reranker_model,
        "llm_model": config.llm_model,
        "num_documents": len(documents),
        "num_chunks": len(chunks)
    },
    "queries": results
}

# Save as JSON
report_json = json.dumps(report, indent=2)
with open('/content/realm_compliance_report.json', 'w') as f:
    f.write(report_json)

print("✅ Report saved to: /content/realm_compliance_report.json")
print("\n📄 Report Summary:")
print(f"   Total Queries: {len(results)}")
print(f"   Documents Indexed: {len(documents)}")
print(f"   Chunks Created: {len(chunks)}")

# =====================================================
# CELL 19: Advanced - Salient Span Masking Demo
# =====================================================
"""
OPTIONAL: Demonstration of REALM's Salient Span Masking pre-training.
This is conceptual - full REALM pre-training requires significant compute.

In production, you would:
1. Mask important regulatory terms
2. Train retriever to find documents that help unmask
3. This teaches the retriever domain expertise
"""

print("🧪 REALM SALIENT SPAN MASKING DEMO")
print("="*80)

example_text = """The [MASK] must be inspected every 500 flight hours according to EASA Part-145."""

print("\nOriginal (Masked) Text:")
print(f"  {example_text}")

print("\nREALM Training Objective:")
print("  The retriever must learn to retrieve the document that contains")
print("  'fire suppression bottle' to fill the [MASK].")

print("\n💡 Why This Matters:")
print("  - Forces retriever to understand domain-specific relationships")
print("  - Unlike standard embeddings, it learns that '500 flight hours'")
print("    is semantically linked to specific maintenance procedures")
print("  - This is why REALM excels at technical/regulatory retrieval")

# =====================================================
# CELL 20: System Summary and Next Steps
# =====================================================
"""
Summary of the REALM RAG pipeline and recommendations.
"""

print("\n" + "="*80)
print("🎯 REALM RAG SYSTEM SUMMARY")
print("="*80)

print("""
✅ **What We Built:**
   - REALM-inspired RAG pipeline for aviation compliance
   - Two-stage retrieval: Dense (MIPS) + Cross-encoder reranking
   - Multi-jurisdictional analysis (FAA, EASA, CAAC)
   - Probabilistic document scoring P(z|x)

📊 **Key Advantages Over Standard RAG:**
   1. Learned Relevance: Reranker understands domain relationships
   2. Better handling of technical terminology (e.g., "N1 vibration")
   3. Cross-document reasoning for regulatory deltas
   4. Confidence scoring for retrieved documents

🚀 **Production Enhancements:**
   1. Add real compliance documents (PDFs from regulatory bodies)
   2. Implement full REALM pre-training with salient span masking
   3. Add citation tracking (show exact regulation sections)
   4. Deploy with FastAPI for web service
   5. Add conversation memory for follow-up questions
   6. Implement regulatory change monitoring (alert on updates)

📚 **To Add Your Own Documents:**
   - Replace synthetic_docs in CELL 4 with PyPDFLoader
   - Upload PDFs to Colab: Files → Upload
   - Load with: PyPDFLoader("/content/your_file.pdf").load()

🔧 **To Improve Retrieval:**
   - Increase top_k_retrieval for broader search
   - Use domain-specific embedding models (legal-bert, etc.)
   - Fine-tune reranker on aviation compliance Q&A pairs
""")

print("\n" + "="*80)
print("✨ REALM RAG Pipeline Ready for Queries! ✨")
print("="*80)

print("\nUse: query_realm_system('Your question here') to test the system")